<h1><center>CMPE 462 - Project 2<br>Implementing an SVM Classifier<br>Due: May 18, 2020, 23:59</center></h1>

* **Student ID1:**
* **Student ID2:**
* **Student ID3:**

## Overview

In this project, you are going to implement SVM. For this purpose, a data set (data.mat) is given to you. You can load the mat dataset into Python using the function `loadmat` in `Scipy.io`. When you load the data, you will obtain a dictionary object, where `X` stores the data matrix and `Y` stores the labels. You can use the first 150 samples for training and the rest for testing. In this project, you will use the software package [`LIBSVM`](http://www.csie.ntu.edu.tw/~cjlin/libsvm/) to implement SVM. Note that `LIBSVM` has a [`Python interface`](https://github.com/cjlin1/libsvm/tree/master/python), so you can call the SVM functions in Python. 

In [1]:
!pip3 install libsvm
!pip3 install PTable

In [2]:
import scipy.io as spio
from libsvm.svmutil import *
from libsvm.svm import *
import numpy as np
from prettytable import PrettyTable

dic = spio.loadmat("data.mat")

train_matrix, train_labels = dic['X'][:150], dic['Y'][:150]
test_matrix, test_labels = dic['X'][150:], dic['Y'][150:]

train_labels = train_labels.reshape((train_labels.shape[0],))
test_labels = test_labels.reshape((test_labels.shape[0],))

## Task 1 - 30 pts

Train a hard margin linear SVM and report both train and test classification accuracy.

In [3]:
def svm_function(params):
    global train_labels, train_matrix, test_labels, test_matrix
    m = svm_train(train_labels, train_matrix, params)
    
    result = []
    p_label, p_acc, p_val = svm_predict(train_labels, train_matrix, m, '-q')
    result.append(str(round(p_acc[0], 4)))
    
    p_label, p_acc, p_val = svm_predict(test_labels, test_matrix, m, '-q')
    result.append(str(round(p_acc[0], 4)))
    return result

In [4]:
res = svm_function('-t 0 -c 100000000')
print('Hard Margin SVM')
print('Train classification accuracy: ' + res[0])
print('Test classification accuracy: ' + res[1])

Hard Margin SVM
Train classification accuracy: 74.6667
Test classification accuracy: 77.5


## Task 2 - 40 pts

Train soft margin SVM for different values of the parameter $C$, and with different kernel functions. Systematically report your results. For instance, report the performances of different kernels for a fixed $C$, then report the performance for different $C$ values for a fixed kernel, and so on.

## Fixed Kernel, Different C-values

In [41]:
# Radial basis function: exp(-gamma*|u-v|^2)
c_v = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
results = []

for c in c_v:
    results.append(svm_function(f'-t 2 -g 1 -c {c}'))

table = PrettyTable()
table.title = 'Kernel -> Radial basis function : exp(-1*|u-v|^2)'
table.field_names = ["C Value"]+[str(c) for c in c_v]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+------------------------------------------------------------------------------------------------+
|                       Kernel -> Radial basis function : exp(-1*|u-v|^2)                        |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+
|    C Value     |  0.001  |   0.01  |   0.1   |    1    |    10   |   100   |   1000  |  10000  |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+
| Train Accuracy | 53.3333 | 53.3333 | 53.3333 |   98.0  |  100.0  |  100.0  |  100.0  |  100.0  |
| Test Accuracy  | 58.3333 | 58.3333 | 58.3333 | 78.3333 | 73.3333 | 73.3333 | 73.3333 | 73.3333 |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+


In [42]:
# Kernel: Polynomial: (gamma*u\'*v)^2

c_v = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
results = []

for c in c_v:
    results.append(svm_function(f'-t 1 -g 1 -r 0 -d 2 -c {c}'))

table = PrettyTable()
table.title ='Kernel -> Polynomial: (u\'*v)^2'

table.field_names = ["C Value"]+[str(c) for c in c_v]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+---------------------------------------------------------------------------------------------+
|                                Kernel -> Polynomial: (u'*v)^2                               |
+----------------+---------+------+---------+---------+---------+---------+---------+---------+
|    C Value     |  0.001  | 0.01 |   0.1   |    1    |    10   |   100   |   1000  |  10000  |
+----------------+---------+------+---------+---------+---------+---------+---------+---------+
| Train Accuracy | 82.6667 | 86.0 |   94.0  | 97.3333 | 99.3333 |  100.0  |  100.0  |  100.0  |
| Test Accuracy  | 83.3333 | 77.5 | 78.3333 |   77.5  |   72.5  | 69.1667 | 69.1667 | 69.1667 |
+----------------+---------+------+---------+---------+---------+---------+---------+---------+


In [43]:
# Kernel: Sigmoid tanh(gamma*u\'*v)

c_v = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
results = []

for c in c_v:
    results.append(svm_function(f'-t 3 -g 1 -r 0 -c {c}'))

table = PrettyTable()
table.title ='Kernel -> Sigmoid: tanh(u\'*v)'

table.field_names = ["C Value"]+[str(c) for c in c_v]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+------------------------------------------------------------------------------------------------+
|                                 Kernel -> Sigmoid: tanh(u'*v)                                  |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+
|    C Value     |  0.001  |   0.01  |   0.1   |    1    |    10   |   100   |   1000  |  10000  |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+
| Train Accuracy | 53.3333 | 59.3333 | 73.3333 |   66.0  | 64.6667 |   64.0  |   64.0  |   64.0  |
| Test Accuracy  | 58.3333 | 65.8333 | 76.6667 | 65.8333 | 64.1667 | 64.1667 | 64.1667 | 64.1667 |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+


In [44]:
# Kernel: Linear: u\'*v

c_v = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
results = []

for c in c_v:
    results.append(svm_function(f'-t 0 -c {c}'))

table = PrettyTable()
table.title ='Kernel -> Linear: u\'*v'

table.field_names = ["C Value"]+[str(c) for c in c_v]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+------------------------------------------------------------------------------------------------+
|                                     Kernel -> Linear: u'*v                                     |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+
|    C Value     |  0.001  |   0.01  |   0.1   |    1    |    10   |   100   |   1000  |  10000  |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+
| Train Accuracy | 53.3333 | 82.6667 |   86.0  | 86.6667 | 88.6667 | 88.6667 |   90.0  |   90.0  |
| Test Accuracy  | 58.3333 | 84.1667 | 83.3333 |   85.0  | 81.6667 | 81.6667 | 81.6667 | 81.6667 |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+


In [45]:
# Kernel: Polynomial: (u\'v)^3


c_v = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
results = []

for c in c_v:
    results.append(svm_function(f'-t 1 -g 1 -r 0 -d 3 -c {c}'))

table = PrettyTable()
table.title ='Kernel: Polynomial -> (u\'*v)^3'

table.field_names = ["C Value"]+[str(c) for c in c_v]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+------------------------------------------------------------------------------------------------+
|                                 Kernel: Polynomial -> (u'*v)^3                                 |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+
|    C Value     |  0.001  |   0.01  |   0.1   |    1    |    10   |   100   |   1000  |  10000  |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+
| Train Accuracy | 88.6667 |   96.0  | 99.3333 |  100.0  |  100.0  |  100.0  |  100.0  |  100.0  |
| Test Accuracy  |   82.5  | 81.6667 | 74.1667 | 75.8333 | 75.8333 | 75.8333 | 75.8333 | 75.8333 |
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+


## Fixed C, different kernels

In [46]:
kernels = ['-t 1 -g 1 -r 0 -d 3 ', '-t 1 -g 1 -r 0 -d 2 ', '-t 0 ', '-t 3 -g 1 -r 0 ', '-t 2 -g 1 '] 
kernel_names = ['polynomial(U\'V)^3', 'polynomial(U\'V)^2', 'linear', 'sigmoid', 'RBF']
results = []

for kernel in kernels:
    results.append(svm_function(f'{kernel}-c 0.01'))

table = PrettyTable()
table.title ='Fixed C = 0.01'

table.field_names = ["Kernel"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                    Fixed C = 0.01                                    |
+----------------+-------------------+-------------------+---------+---------+---------+
|     Kernel     | polynomial(U'V)^3 | polynomial(U'V)^2 |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |        96.0       |        86.0       | 82.6667 | 59.3333 | 53.3333 |
| Test Accuracy  |      81.6667      |        77.5       | 84.1667 | 65.8333 | 58.3333 |
+----------------+-------------------+-------------------+---------+---------+---------+


In [47]:
kernels = ['-t 1 -g 1 -r 0 -d 3 ', '-t 1 -g 1 -r 0 -d 2 ', '-t 0 ', '-t 3 -g 1 -r 0 ', '-t 2 -g 1 '] 
kernel_names = ['polynomial(U\'V)^3', 'polynomial(U\'V)^2', 'linear', 'sigmoid', 'RBF']
results = []

for kernel in kernels:
    results.append(svm_function(f'{kernel}-c 0.1'))

table = PrettyTable()
table.title ='Fixed C = 0.1'

table.field_names = ["Kernel"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                    Fixed C = 0.1                                     |
+----------------+-------------------+-------------------+---------+---------+---------+
|     Kernel     | polynomial(U'V)^3 | polynomial(U'V)^2 |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |      99.3333      |        94.0       |   86.0  | 73.3333 | 53.3333 |
| Test Accuracy  |      74.1667      |      78.3333      | 83.3333 | 76.6667 | 58.3333 |
+----------------+-------------------+-------------------+---------+---------+---------+


In [48]:
kernels = ['-t 1 -g 1 -r 0 -d 3 ', '-t 1 -g 1 -r 0 -d 2 ', '-t 0 ', '-t 3 -g 1 -r 0 ', '-t 2 -g 1 '] 
kernel_names = ['polynomial(U\'V)^3', 'polynomial(U\'V)^2', 'linear', 'sigmoid', 'RBF']
results = []

for kernel in kernels:
    results.append(svm_function(f'{kernel}-c 1'))

table = PrettyTable()
table.title ='Fixed C = 1'

table.field_names = ["Kernel"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                     Fixed C = 1                                      |
+----------------+-------------------+-------------------+---------+---------+---------+
|     Kernel     | polynomial(U'V)^3 | polynomial(U'V)^2 |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |       100.0       |      97.3333      | 86.6667 |   66.0  |   98.0  |
| Test Accuracy  |      75.8333      |        77.5       |   85.0  | 65.8333 | 78.3333 |
+----------------+-------------------+-------------------+---------+---------+---------+


In [49]:
kernels = ['-t 1 -g 1 -r 0 -d 3 ', '-t 1 -g 1 -r 0 -d 2 ', '-t 0 ', '-t 3 -g 1 -r 0 ', '-t 2 -g 1 '] 
kernel_names = ['polynomial(U\'V)^3', 'polynomial(U\'V)^2', 'linear', 'sigmoid', 'RBF']
results = []

for kernel in kernels:
    results.append(svm_function(f'{kernel}-c 10'))

table = PrettyTable()
table.title ='Fixed C = 10'

table.field_names = ["Kernel"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                     Fixed C = 10                                     |
+----------------+-------------------+-------------------+---------+---------+---------+
|     Kernel     | polynomial(U'V)^3 | polynomial(U'V)^2 |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |       100.0       |      99.3333      | 88.6667 | 64.6667 |  100.0  |
| Test Accuracy  |      75.8333      |        72.5       | 81.6667 | 64.1667 | 73.3333 |
+----------------+-------------------+-------------------+---------+---------+---------+


In [50]:
kernels = ['-t 1 -g 1 -r 0 -d 3 ', '-t 1 -g 1 -r 0 -d 2 ', '-t 0 ', '-t 3 -g 1 -r 0 ', '-t 2 -g 1 '] 
kernel_names = ['polynomial(U\'V)^3', 'polynomial(U\'V)^2', 'linear', 'sigmoid', 'RBF']
results = []

for kernel in kernels:
    results.append(svm_function(f'{kernel}-c 100'))

table = PrettyTable()
table.title ='Fixed C = 100'

table.field_names = ["Kernel"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                    Fixed C = 100                                     |
+----------------+-------------------+-------------------+---------+---------+---------+
|     Kernel     | polynomial(U'V)^3 | polynomial(U'V)^2 |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |       100.0       |       100.0       | 88.6667 |   64.0  |  100.0  |
| Test Accuracy  |      75.8333      |      69.1667      | 81.6667 | 64.1667 | 73.3333 |
+----------------+-------------------+-------------------+---------+---------+---------+


In [51]:
kernels = ['-t 1 -g 1 -r 0 -d 3 ', '-t 1 -g 1 -r 0 -d 2 ', '-t 0 ', '-t 3 -g 1 -r 0 ', '-t 2 -g 1 '] 
kernel_names = ['polynomial(U\'V)^3', 'polynomial(U\'V)^2', 'linear', 'sigmoid', 'RBF']
results = []

for kernel in kernels:
    results.append(svm_function(f'{kernel}-c 1000'))

table = PrettyTable()
table.title ='Fixed C = 1000'

table.field_names = ["Kernel"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                    Fixed C = 1000                                    |
+----------------+-------------------+-------------------+---------+---------+---------+
|     Kernel     | polynomial(U'V)^3 | polynomial(U'V)^2 |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |       100.0       |       100.0       |   90.0  |   64.0  |  100.0  |
| Test Accuracy  |      75.8333      |      69.1667      | 81.6667 | 64.1667 | 73.3333 |
+----------------+-------------------+-------------------+---------+---------+---------+


## Task 3 - 15 pts

Please report how the number of support vectors changes as the value of $C$ increases (while all other parameters remain the same). Discuss whether your observations match the theory.

In [52]:
def get_number_of_sv(params):
    global train_labels, train_matrix
    m = svm_train(train_labels, train_matrix, params)
    return m.get_nr_sv()

In [53]:
c_values = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
num_sv = []
for c in c_values:
    num_sv.append(get_number_of_sv(f'-t 0 -c {c}'))
    
table = PrettyTable()

table.title ='Kernel -> Linear: u\'*v'

table.field_names = ["C Value"]+[str(c) for c in c_values]
table.add_row(["Number of Support Vectors"]+num_sv)
print(table)

+-------------------------------------------------------------------------------+
|                             Kernel -> Linear: u'*v                            |
+---------------------------+-------+------+-----+----+----+-----+------+-------+
|          C Value          | 0.001 | 0.01 | 0.1 | 1  | 10 | 100 | 1000 | 10000 |
+---------------------------+-------+------+-----+----+----+-----+------+-------+
| Number of Support Vectors |  140  | 118  |  74 | 58 | 51 |  50 |  49  |   49  |
+---------------------------+-------+------+-----+----+----+-----+------+-------+


## Task 4 - 15 pts

Please investigate the changes in the hyperplane when you remove one of the support vectors, vs., one data point that is not a support vector.

In [18]:
def get_weight(data,m):
    
    SV_coef = np.zeros(shape=(m.get_nr_sv()))
    SVs = np.zeros(shape=(m.get_nr_sv(),data.shape[1]))

    for index in range(0,m.get_nr_sv()):
        SV_coef[index] = m.get_sv_coef()[index][0]
        SVs[index] = data[m.get_sv_indices()[index]-1]
        
        
    weight = np.dot(SV_coef.transpose(),SVs)

    return weight, m.get_nr_sv() ,np.linalg.norm(weight)

In [19]:
def delete_sv(data,labels,m,param):
    new_set = np.delete(data,m.get_sv_indices()[0]-1,0)
    new_labels = np.delete(labels,m.get_sv_indices()[0]-1,0)
    m = svm_train(new_labels,new_set,param)
    weight, Number_Of_Svs, w_norm = get_weight(new_set,m)
   
    return weight,Number_Of_Svs ,w_norm 

In [20]:
def delete_dp(data,labels,m,param):
    comparison = np.arange(data.shape[0])+1
    sv_indices = m.get_sv_indices()
    
    weight=0
    Number_Of_Svs=0
    w_norm = 0

    for index in comparison:
        if index not in sv_indices:
            
            new_set = np.delete(data,index-1,0)
            new_labels = np.delete(labels,index-1,0)
            m = svm_train(new_labels,new_set,param)
            weight,Number_Of_Svs, w_norm= get_weight(new_set,m)
            break
            
    return weight, Number_Of_Svs , w_norm

In [21]:
param = '-t 0 -c 1'
m = svm_train(train_labels,train_matrix,param)

weight , Number_Of_Svs ,w_norm = get_weight(train_matrix,m)

print(f"Weight: {weight}")
print(f"Norm of the weight: {w_norm}")
print(f"Number of Support Vectors: {Number_Of_Svs}")

Weight: [-0.47637751  0.39813231  0.86713369  0.57366213  0.32304567  0.10008082
  0.05139059 -0.95512925  0.07086555  0.00576303  0.24894908  1.82218655
  0.5129991 ]
Norm of the weight: 2.4791740955745105
Number of Support Vectors: 58


### Removing one data point that is not a support vector

In [22]:
#removing one data point that is not a support vector.
weight , Number_Of_Svs ,w_norm= delete_dp(train_matrix,train_labels,m,param)

print(f"Weight: {weight}")
print(f"Norm of the weight: {w_norm}")
print(f"Number of Support Vectors: {Number_Of_Svs}")

Weight: [-0.47637751  0.39813231  0.86713369  0.57366213  0.32304567  0.10008082
  0.05139059 -0.95512925  0.07086555  0.00576303  0.24894908  1.82218655
  0.5129991 ]
Norm of the weight: 2.4791740955745105
Number of Support Vectors: 58


### Removing one of the support vectors

In [23]:
#removing one of the support vectors
weight , Number_Of_Svs,w_norm = delete_sv(train_matrix,train_labels,m,param)

print(f"Weight: {weight}")
print(f"Norm of the weight: {w_norm}")
print(f"Number of Support Vectors: {Number_Of_Svs}")

Weight: [-0.66183675  0.36149846  0.93849842  0.57538557  0.36977623  0.10905087
  0.07963851 -0.98344676  0.02084041  0.09060237  0.23487472  1.9661612
  0.51400789]
Norm of the weight: 2.6639250107862242
Number of Support Vectors: 56


### Bonus Task - 10 pts

Use Python and [CVXOPT](http://cvxopt.org) QP solver to implement the hard margin SVM. 

In [24]:
def get_accuracy(data,labels,w,b):
    
    result = np.sign(np.dot(data, w) +b).astype(int)
    mul = np.multiply(result, labels)
    acc = np.count_nonzero(mul == 1) / len(labels)
    return acc*100

In [25]:
from cvxopt import matrix, solvers
X = np.array([[0, 0], [2, 2],[2,0],[3,0]])
y = np.array([-1, -1, 1, 1])

sp_v_threshold = 1e-8
n ,dim = X.shape
# initialize variables
Q = matrix(np.identity(dim + 1, dtype=np.float))
p = matrix(np.zeros((dim + 1,), dtype=np.float))
A = matrix(np.zeros((n, dim + 1), dtype=np.float))
c = -matrix(np.ones((n,), dtype=np.float))
Q[0, 0] = 0

for i in range(n):
    A[i, 0] = -y[i]* 1.
    A[i, 1:] = -y[i]*X[i, :]* 1.

    
sol = solvers.qp(Q,p,A,c)

w = np.zeros(dim,) # weight
b = sol["x"][0] # bias

for i in range(1, dim + 1):
     w[i - 1] = sol["x"][i]
    

print(f"weight: {w}")
print(f"bias: {b}")
train_accuracy = get_accuracy(X,y,w,b)
print(f"Train classification accuracy: {train_accuracy}")
#print(y)
#Sp_v = []


#find the indexes of support vectors
# for i in range(n):
#     v = y[i] * (np.dot(X[i],w) + b)
#     print(v)
#     if v < (1 + sp_v_threshold):
#         Sp_v.append(X[i])
        
# print(Sp_v)

     pcost       dcost       gap    pres   dres
 0:  3.2653e-01  1.9592e+00  6e+00  2e+00  4e+00
 1:  1.5796e+00  8.5663e-01  7e-01  2e-16  1e-15
 2:  1.0195e+00  9.9227e-01  3e-02  4e-16  9e-16
 3:  1.0002e+00  9.9992e-01  3e-04  2e-16  2e-15
 4:  1.0000e+00  1.0000e+00  3e-06  2e-16  1e-15
 5:  1.0000e+00  1.0000e+00  3e-08  1e-16  8e-16
Optimal solution found.
weight: [ 1.00000001 -1.00000001]
bias: -1.0000000134075104
Train classification accuracy: 100.0
